<a href="https://colab.research.google.com/github/manognadeva/Makeup-Recommender/blob/main/SLAY_Project_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
base_path = "/content/drive/MyDrive/fairface/fairface/"
train_csv_path = base_path + "fitz_undersampled_train_final.csv"
test_csv_path = base_path + "fitz_undersampled_test_final.csv"
train_image_dir = base_path + "train/"
test_image_dir = base_path + "val/"

In [18]:
 import numpy as np
import os
import pandas as pd
from PIL import Image
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential, mixed_precision
from tensorflow.keras.layers import (
    RandomFlip,
    RandomRotation,
    RandomZoom,
    Dense,
    Flatten,
    Dropout,
    GlobalAveragePooling2D,
    BatchNormalization
)
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.mixed_precision import LossScaleOptimizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from flask import Flask, render_template, request, jsonify
from pyngrok import ngrok

In [4]:
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

train_df.head()

,file,age,gender,race,phototype
0,1.jpg,50-59,Male,East Asian,III
1,10.jpg,30-39,Male,Middle Eastern,V
2,100.jpg,20-29,Female,East Asian,III
3,1000.jpg,30-39,Male,White,I & II
4,10004.jpg,40-49,Male,Indian,V


In [5]:
phototype_mapping = {
    "I": "Light", "II": "Light",
    "III": "Medium", "IV": "Medium",
    "V": "Dark", "VI": "Dark",
    "I & II": "Light", "III & IV": "Medium", "V & VI": "Dark"
}

train_df['skin_tone'] = train_df['phototype'].map(phototype_mapping)
test_df['skin_tone'] = test_df['phototype'].map(phototype_mapping)

train_df['file_path'] = train_df['file'].apply(lambda x: os.path.join(train_image_dir, x))
test_df['file_path'] = test_df['file'].apply(lambda x: os.path.join(test_image_dir, x))

train_df.head()

,file,age,gender,race,phototype,skin_tone,file_path
0,1.jpg,50-59,Male,East Asian,III,Medium,/content/drive/MyDrive/fairface/fairface/train...
1,10.jpg,30-39,Male,Middle Eastern,V,Dark,/content/drive/MyDrive/fairface/fairface/train...
2,100.jpg,20-29,Female,East Asian,III,Medium,/content/drive/MyDrive/fairface/fairface/train...
3,1000.jpg,30-39,Male,White,I & II,Light,/content/drive/MyDrive/fairface/fairface/train...
4,10004.jpg,40-49,Male,Indian,V,Dark,/content/drive/MyDrive/fairface/fairface/train...


In [6]:
train_df['file_exists'] = train_df['file_path'].apply(os.path.exists)
train_df = train_df[train_df['file_exists']]

print(f"Updated dataset size: {len(train_df)}")

Updated dataset size: 35424


In [7]:
def preprocess_data(train_df, test_df, BATCH_SIZE=32, IMAGE_SIZE=(96, 96)):
    def load_image(file_path, label):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, IMAGE_SIZE)
        return img, label

    train_labels = pd.get_dummies(train_df['skin_tone']).values
    test_labels = pd.get_dummies(test_df['skin_tone']).values

    train_dataset = tf.data.Dataset.from_tensor_slices(
        (train_df['file_path'].values, train_labels)
    )
    test_dataset = tf.data.Dataset.from_tensor_slices(
        (test_df['file_path'].values, test_labels)
    )

    train_dataset = (train_dataset
        .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
        .cache()
        .shuffle(1000)
        .batch(BATCH_SIZE)
        .prefetch(tf.data.AUTOTUNE)
        .repeat())

    test_dataset = (test_dataset
        .map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
        .cache()
        .batch(BATCH_SIZE)
        .prefetch(tf.data.AUTOTUNE)
        .repeat())
    return train_dataset, test_dataset

In [8]:
def create_model(IMAGE_SIZE=(96, 96)):
    base_model = tf.keras.applications.ResNet50V2(
        weights='imagenet',
        include_top=False,
        input_shape=(*IMAGE_SIZE, 3)
    )

    for layer in base_model.layers[:-30]:
        layer.trainable = False

    model = Sequential([
        tf.keras.layers.Rescaling(1./255),

        tf.keras.layers.RandomBrightness(0.2),
        tf.keras.layers.RandomContrast(0.1),
        tf.keras.layers.RandomFlip("horizontal"),

        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dense(64, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        Dense(3, activation='softmax', dtype='float32')
    ])

    optimizer = Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [9]:
def train_model(model, train_dataset, test_dataset, train_df, test_df, BATCH_SIZE=32):
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            min_delta=0.01
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-6,
            verbose=1
        ),
        ModelCheckpoint(
            'best_model.keras',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]

    steps_per_epoch = len(train_df) // BATCH_SIZE
    validation_steps = len(test_df) // BATCH_SIZE

    history = model.fit(
        train_dataset,
        epochs=5,
        validation_data=test_dataset,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        callbacks=callbacks,
        shuffle=True
    )

    return history

In [11]:
# # Preprocess data
# train_dataset, test_dataset = preprocess_data(train_df, test_df)

# # Create and train model
# model = create_model()
# history = train_model(model, train_dataset, test_dataset, train_df, test_df)

Epoch 1/5
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.4442 - loss: 1.3647
Epoch 1: val_accuracy improved from -inf to 0.39960, saving model to best_model.keras
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 11286s 10s/step - accuracy: 0.4442 - loss: 1.3646 - val_accuracy: 0.3996 - val_loss: 1.5500 - learning_rate: 1.0000e-04
Epoch 2/5
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4856 - loss: 1.2541
Epoch 2: val_accuracy did not improve from 0.39960
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 2367s 2s/step - accuracy: 0.4856 - loss: 1.2541 - val_accuracy: 0.3957 - val_loss: 1.5597 - learning_rate: 1.0000e-04
Epoch 3/5
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4875 - loss: 1.2310
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 3: val_accuracy did not improve from 0.39960
1107/1107 ━━━━━━━━━━━━━━━━━━━━ 2320s 2s/step - accuracy: 0.4875 - loss: 1.2310 - val_accuracy: 0.3991 - val_loss: 1.5630 - learning_rate: 1.0000e-04
Epoch 4/5
1107/1107 ━━━━━━━━

In [21]:
def predict_and_recommend_makeup(model, image_path, choice1, choice2, IMAGE_SIZE=(96, 96)):
    makeup_recommendations = {
        'Light': {
            'Party': {
                'eyeshadow': 'Champagne Shimmer',
                'blush': 'Baby Pink',
                'lipstick': 'Light Rose Pink'
            },
            'No Makeup': {
                'eyeshadow': 'Soft Pink',
                'blush': 'Pale Peach',
                'lipstick': 'Peachy Nude'
            }
        },
        'Medium': {
            'Party': {
                'eyeshadow': 'Copper Shimmer',
                'blush': 'Rosy Pink',
                'lipstick': 'Rosy Brown'
            },
            'No Makeup': {
                'eyeshadow': 'Warm Bronze',
                'blush': 'Warm Peach',
                'lipstick': 'Caramel Nude'
            }
        },
        'Dark': {
            'Party': {
                'eyeshadow': 'Bold Gold',
                'blush': 'Deep Rose',
                'lipstick': 'Dark Cherry Red'
            },
            'No Makeup': {
                'eyeshadow': 'Deep Copper',
                'blush': 'Warm Terracotta',
                'lipstick': 'Rich Chocolate Brown'
            }
        }
    }

    valid_categories = ['eyeshadow', 'blush', 'lipstick']
    if choice1.lower() not in valid_categories or choice2.lower() not in valid_categories:
        return "Please choose two valid categories from: eyeshadow, blush, lipstick."
    if choice1.lower() == choice2.lower():
        return "Please choose two different categories."

    # Preprocess the image
    try:
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
    except Exception as e:
        return f"Error processing the image: {str(e)}"

    # Predict the skin tone
    prediction = model.predict(img_array)
    skin_tones = ['Light', 'Medium', 'Dark']
    predicted_skin_tone = skin_tones[np.argmax(prediction)]

    # Results
    result = f"Skin Tone: {predicted_skin_tone}\n\n"

    result += "Party Look:\n"
    result += f"• {choice1.capitalize()}: {makeup_recommendations[predicted_skin_tone]['Party'][choice1]}\n"
    result += f"• {choice2.capitalize()}: {makeup_recommendations[predicted_skin_tone]['Party'][choice2]}\n\n"

    result += "No Makeup Look:\n"
    result += f"• {choice1.capitalize()}: {makeup_recommendations[predicted_skin_tone]['No Makeup'][choice1]}\n"
    result += f"• {choice2.capitalize()}: {makeup_recommendations[predicted_skin_tone]['No Makeup'][choice2]}"

    return result

In [22]:
model = tf.keras.models.load_model('/content/best_model.keras')

result = predict_and_recommend_makeup(model, '/content/input.jpg', 'eyeshadow', 'blush')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Skin Tone: Light

Party Look:
• Eyeshadow: Champagne Shimmer
• Blush: Baby Pink

No Makeup Look:
• Eyeshadow: Soft Pink
• Blush: Pale Peach


In [17]:
!pip install flask-ngrok
!pip install pyngrok
!pip install flask
!ngrok config add-authtoken 2qtvG7lXqoya1uK7dNaSr6v6mvq_4hxiLsaniRm6gSkvfsC6w

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Install required libraries
!pip install flask pyngrok tensorflow

# Import necessary modules
from flask import Flask, request, render_template, url_for
from pyngrok import ngrok
from werkzeug.utils import secure_filename
import os
import tensorflow as tf
import numpy as np

# Initialize Flask app
app = Flask(__name__)

# Set up static folder for uploads
STATIC_FOLDER = os.path.join('static', 'uploads')
if not os.path.exists(STATIC_FOLDER):
    os.makedirs(STATIC_FOLDER)
app.config['UPLOAD_FOLDER'] = STATIC_FOLDER

# Set up ngrok
ngrok.set_auth_token("2qtvG7lXqoya1uK7dNaSr6v6mvq_4hxiLsaniRm6gSkvfsC6w")

def predict_and_recommend_makeup(model, image_path, choice1, choice2, IMAGE_SIZE=(96, 96)):
    makeup_recommendations = {
        'Light': {
            'Party': {
                'eyeshadow': 'Champagne Shimmer',
                'blush': 'Baby Pink',
                'lipstick': 'Light Rose Pink'
            },
            'No Makeup': {
                'eyeshadow': 'Soft Pink',
                'blush': 'Pale Peach',
                'lipstick': 'Peachy Nude'
            }
        },
        'Medium': {
            'Party': {
                'eyeshadow': 'Copper Shimmer',
                'blush': 'Rosy Pink',
                'lipstick': 'Rosy Brown'
            },
            'No Makeup': {
                'eyeshadow': 'Warm Bronze',
                'blush': 'Warm Peach',
                'lipstick': 'Caramel Nude'
            }
        },
        'Dark': {
            'Party': {
                'eyeshadow': 'Bold Gold',
                'blush': 'Deep Rose',
                'lipstick': 'Dark Cherry Red'
            },
            'No Makeup': {
                'eyeshadow': 'Deep Copper',
                'blush': 'Warm Terracotta',
                'lipstick': 'Rich Chocolate Brown'
            }
        }
    }

    valid_categories = ['eyeshadow', 'blush', 'lipstick']
    if choice1.lower() not in valid_categories or choice2.lower() not in valid_categories:
        return "Please choose two valid categories from: eyeshadow, blush, lipstick."
    if choice1.lower() == choice2.lower():
        return "Please choose two different categories."

    try:
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
    except Exception as e:
        return f"Error processing the image: {str(e)}"

    prediction = model.predict(img_array)
    skin_tones = ['Light', 'Medium', 'Dark']
    predicted_skin_tone = skin_tones[np.argmax(prediction)]

    result = f"Skin Tone: {predicted_skin_tone}\n\n"
    result += "Party Look:\n"
    result += f"• {choice1.capitalize()}: {makeup_recommendations[predicted_skin_tone]['Party'][choice1]}\n"
    result += f"• {choice2.capitalize()}: {makeup_recommendations[predicted_skin_tone]['Party'][choice2]}\n\n"
    result += "No Makeup Look:\n"
    result += f"• {choice1.capitalize()}: {makeup_recommendations[predicted_skin_tone]['No Makeup'][choice1]}\n"
    result += f"• {choice2.capitalize()}: {makeup_recommendations[predicted_skin_tone]['No Makeup'][choice2]}"

    return result

@app.route('/', methods=['GET'])
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return "No file uploaded", 400

    file = request.files['file']
    if file.filename == '':
        return "No file selected", 400

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        choice1 = request.form.get('choice1')
        choice2 = request.form.get('choice2')

        result = predict_and_recommend_makeup(model, filepath, choice1, choice2)

        # Create relative path for template
        relative_path = os.path.join('uploads', filename)

        return render_template('result.html',
                             result=result,
                             image_path=relative_path)
    else:
        return "Invalid file type", 400

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in {'png', 'jpg', 'jpeg', 'gif'}

if __name__ == '__main__':
    # Start ngrok
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000/\"")

    # Run the Flask app
    app.run()


 * ngrok tunnel "NgrokTunnel: "https://2642-35-229-204-95.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000/"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:18:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:18:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:18:24] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:18:32] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:18:33] "GET /static/uploads/Image.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:18:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:18:43] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:19:12] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:19:12] "GET /static/uploads/Image.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:23:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:23:39] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:24:02] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jan/2025 21:24:02] "GET /static/uploads/Image.jpg HTTP/1.1" 200 -
